In [4]:
import os
import sys
import pandas as pd
import numpy as np
import neptune
from neptune.utils import stringify_unsupported
from utils_neptune import start_neptune_run

In [3]:

import sklearn
print('sklearn version: ', sklearn.__version__)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

sklearn version:  1.3.2


In [2]:
data_dir ='/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/alignment_RCC_2024_Feb_27/March_22_Data'

In [15]:
X_train = pd.read_csv(os.path.join(data_dir, 'X_finetune_train.csv'),index_col=0)
y_train = pd.read_csv(os.path.join(data_dir, 'y_finetune_train.csv'),index_col=0)
X_val = pd.read_csv(os.path.join(data_dir, 'X_finetune_val.csv'), index_col=0)
y_val = pd.read_csv(os.path.join(data_dir, 'y_finetune_val.csv'), index_col=0)

In [16]:
y_val = y_val['MSKCC BINARY']
y_train = y_train['MSKCC BINARY']

# drop the nans
y_val = y_val.dropna()
X_val = X_val.loc[y_val.index]
y_train = y_train.dropna()
X_train = X_train.loc[y_train.index]

In [26]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_prob = lr.predict_proba(X_val)[:,1]


train_roc_auc = roc_auc_score(y_train, lr.predict_proba(X_train)[:,1], average='micro')
val_roc_auc = roc_auc_score(y_val, y_prob, average='micro')

run, _ = start_neptune_run(with_run_id=None,tags=['v3.1','classical'])
run['finetune_mkscc/kwargs/model_kind'] ='LogisticRegression'
run['finetune_mkscc/eval/train/Binary_MSKCC/AUROC (micro)'] = train_roc_auc
run['finetune_mkscc/eval/val/Binary_MSKCC/AUROC (micro)'] = val_roc_auc

run.stop()

In [27]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_prob = rf.predict_proba(X_val)[:,1]

train_roc_auc = roc_auc_score(y_train, rf.predict_proba(X_train)[:,1], average='micro')
val_roc_auc = roc_auc_score(y_val, y_prob, average='micro')

run, _ = start_neptune_run(with_run_id=None,tags=['v3.1','classical'])
run['finetune_mkscc/kwargs/model_kind'] ='RandomForestClassifier'
run['finetune_mkscc/eval/train/Binary_MSKCC/AUROC (micro)'] = train_roc_auc
run['finetune_mkscc/eval/val/Binary_MSKCC/AUROC (micro)'] = val_roc_auc

run.stop()


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-1426
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 3 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-1426/metadata


In [28]:
svc = SVC(probability=True)
svc.fit(X_train, y_train)
y_prob = svc.predict_proba(X_val)[:,1]

train_roc_auc = roc_auc_score(y_train, svc.predict_proba(X_train)[:,1], average='micro')
val_roc_auc = roc_auc_score(y_val, y_prob, average='micro')

run, _ = start_neptune_run(with_run_id=None,tags=['v3.1','classical'])
run['finetune_mkscc/kwargs/model_kind'] ='SVC'
run['finetune_mkscc/eval/train/Binary_MSKCC/AUROC (micro)'] = train_roc_auc
run['finetune_mkscc/eval/val/Binary_MSKCC/AUROC (micro)'] = val_roc_auc

run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-1427
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 3 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-1427/metadata
